# 루브릭

1. 데이터의 전처리 및 구성과정이 체계적으로 진행되었는가?
 - 특수문자 제거, 토크나이저 생성, 패딩 처리의 작업들이 빠짐없이 진행되었는가? (0)
2. 가사 텍스트 생성 모델이 정상적으로 동작하는가? 
 - 텍스트 제너레이션 결과로 생성된 문장이 해석 가능한 문장인가? (0)
3. 텍스트 생성모델이 안정적으로 학습되었는가?
 - 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가? (0)

In [1]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow as tf
import os, re 
import numpy as np

print(tf.__version__)

2.6.0


In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = [] 

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['', '', '[Spoken Intro:]']


## 데이터 정제

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    le = preprocessed_sentence.split()
    if len(le) > 17: continue
    corpus.append(preprocessed_sentence)
    
    
len(corpus)

163488

- 앞뒤로 start, end까지 포함하여 문장 수가 17가 넘어가는 문장은 학습데이터에서 제외하였다.

## 평가 데이터셋 분리

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=13000, 
        filters=' ',
        oov_token="<unk>"
    )

    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=20)  

    return tensor, tokenizer

In [6]:
tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

In [7]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=55)

- tokenize() 함수로 데이터를 Tensor로 변환후 src_input, tgt_input로 나눈뒤 train_test_split()을 이용하여 train과 vaildation 데이터로 나눔

## 모델 만들기

In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)

- from_tensor_slices() 메서드를 사용하여 enc_train과 dec_train을 묶는다.
- shuffle() 사용하여 데이터를 랜덤하게 섞는다.
- batch() 메서드를 사용하여 각 데이터셋을 미니배치로 묶는다.
- (drop_remainder=True)는 마지막 배치가 지정된 크기보다 작은 경우 해당 배치를 삭제
- prefetch() 함수를 사용하여 데이터를 미리 가져오도록 설정
- prefetch() 함수에서 AUTOTUNE 옵션을 사용하면, TensorFlow는 데이터를 미리 가져오는 동안 CPU와 GPU를 병렬적으로 사용하여 학습 속도를 높일 수 있다.

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [10]:
embedding_size = 256
hidden_size = 1024
model = TextGenerator(VOCAB_SIZE, embedding_size , hidden_size)

In [11]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, 
    reduction='none'
)

In [12]:
model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, epochs=10, validation_data=val_dataset) 

Epoch 1/10
510/510 [==============================] - 157s 250ms/step - loss: 2.7896 - val_loss: 2.4716
Epoch 2/10
510/510 [==============================] - 134s 262ms/step - loss: 2.3639 - val_loss: 2.2964
Epoch 3/10
510/510 [==============================] - 135s 265ms/step - loss: 2.2275 - val_loss: 2.2101
Epoch 4/10
510/510 [==============================] - 135s 265ms/step - loss: 2.1307 - val_loss: 2.1475
Epoch 5/10
510/510 [==============================] - 135s 265ms/step - loss: 2.0493 - val_loss: 2.0977
Epoch 6/10
510/510 [==============================] - 136s 266ms/step - loss: 1.9771 - val_loss: 2.0579
Epoch 7/10
510/510 [==============================] - 135s 265ms/step - loss: 1.9088 - val_loss: 2.0218
Epoch 8/10
510/510 [==============================] - 135s 265ms/step - loss: 1.8437 - val_loss: 1.9902
Epoch 9/10
510/510 [==============================] - 136s 265ms/step - loss: 1.7807 - val_loss: 1.9640
Epoch 10/10
510/510 [==============================] - 136s 265m

validation loss가 1.9421로 2.2이하로 내려간것을 확인할 수 있다.

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3328256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  13326025  
Total params: 30,293,961
Trainable params: 30,293,961
Non-trainable params: 0
_________________________________________________________________


In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [19]:
ly = ['<start> This night', '<start> I can', '<start> From the', '<start> Your steps', '<start> Don t cut', '<start> Now I m'
      , '<start> Could you', '<start> A little', '<start> If you', '<start> If you re', '<start> Let me']
for i in ly:
    generated = generate_text(model, tokenizer, init_sentence=i, max_len=20)
    print(generated)


<start> this night i m a freak <end> 
<start> i can t help you , i m better <end> 
<start> from the <unk> of the lamb <end> 
<start> your steps are on fire <end> 
<start> don t cut it off <end> 
<start> now i m a voodoo chile <end> 
<start> could you be the same <end> 
<start> a little bit of the night <end> 
<start> if you want me , you re a liar <end> 
<start> if you re a good girl , you better work bitch <end> 
<start> let me see you again <end> 


Let Me Down Slowly - 앨릭 벤저민의 노래가사의 일부의 앞부분만 가져와서 문장을 생성해보았다. <br/>
노래가사와는 당연히 맞지 않지만 거의 문장이 자연스럽게 나온 것을 볼 수 있다.

---
## 회고

 - 처음 해보는 NLP 프로젝트였는데 생각보다 재미있었다.
 - 처음 보는 것들이 있어 이해하는데 시간이 걸렸지만 배운것이 생겨 좋았다.
 - 더 많은 단어와 문장을 가지고 학습을 시켜 모델을 만들면 더 좋게 나올듯하여 궁금해졌다.
 - 생각이상으로 생성한 문장이 자연스러워 놀라웠다.